In [91]:
import pickle as pkl
import time
import os
import numpy as np
import sys
from music21 import instrument, note, stream, chord, duration
from RNNmodel import create_network
import matplotlib.pyplot as plt

In [92]:
with open("dictionary.pkl", "rb") as input_file:
    dictionary = pkl.load(input_file)


durations_to_int = dictionary["durations_to_int"]
int_to_duration = dictionary["int_to_duration"]
notes_to_int = dictionary["notes_to_int"]
int_to_notes = dictionary["int_to_notes"]
phrases_to_int = dictionary["phrases_to_int"]
int_to_phrases = dictionary["int_to_phrases"]

n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)

In [93]:
weights_folder = 'weights'
weights_file = 'weights.h5'
embed_size = 100
rnn_units = 256
use_attention = True

model, att_model = create_network(n_notes, n_durations, n_phrases, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_10 (Embedding)       (None, None, 100)    4400        ['input_11[0][0]']               
                                                                                                  
 embedding_11 (Embedding)       (None, None, 100)    1900        ['input_12[0][0]']               
                                                                                           

In [94]:
import pickle
piece = None
with open('../data/set_1/test/pieces_test.pkl', 'rb') as handle:
    piece = pickle.load(handle)

In [95]:
len(piece)

869

In [96]:
piece[0][2]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [97]:
notes_input = [ notes_to_int[note] for note in piece[0][0] ]
print(notes_input)

[16, 21, 21, 25, 23, 21, 23, 23, 23, 26, 25, 23, 18, 18, 21, 20, 18, 25, 42, 28, 28, 26, 30, 30, 26, 26, 26, 25, 28, 25, 25, 25, 23, 25, 26, 28, 42, 28, 28, 28, 30, 30, 26, 26, 25, 28, 25, 25, 23, 26, 25, 21, 42]


In [98]:
durations_input = [ durations_to_int[duration] for duration in piece[0][1] ]
print(durations_input)

[7, 10, 7, 7, 7, 7, 9, 4, 7, 7, 7, 7, 10, 7, 7, 7, 7, 13, 7, 7, 10, 7, 7, 7, 7, 7, 7, 7, 10, 7, 7, 7, 7, 10, 7, 13, 7, 7, 10, 7, 7, 7, 7, 10, 7, 10, 7, 10, 7, 10, 7, 13, 7]


In [99]:
phrases_input = piece[0][2]
print(phrases_input)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


In [100]:
max_extra_notes = len(notes_input) - 1
seq_len = 5
phrases_output = []
phrases_output.append(0) # after the first note (that is already in the initial sequence)

notes_input_sequence = [notes_input[0]] # the first note element of the sequence
durations_input_sequence = [durations_input[0]] # the first duration element of the sequence

if seq_len is not None:
    notes_input_sequence = [notes_to_int[129]] * (seq_len - len(notes_input_sequence)) + notes_input_sequence # the sequence length must be always equal seq_len, therefore we fill the vector with values 129 (the 'START' token) at the beginning 
    durations_input_sequence = [0] * (seq_len - len(durations_input_sequence)) + durations_input_sequence # the same for durations

In [101]:
print(notes_input_sequence)

[43, 43, 43, 43, 16]


In [102]:
print(durations_input_sequence)

[0, 0, 0, 0, 7]


In [103]:
for note_index in range(max_extra_notes):

    prediction_input = [
        np.array([notes_input_sequence])
        , np.array([durations_input_sequence])
       ]

    phrases_prediction = model.predict(prediction_input, verbose=0)
    print(notes_input_sequence)
    print(durations_input_sequence)
    print(phrases_prediction[0][0])
    print(phrases_prediction[0][1])
    print("===")
    print(phrases_prediction)
    
    phrases_output.append(0 if phrases_prediction[0][0] >= phrases_prediction[0][1] else 1)

    notes_input_sequence.append(notes_input[note_index])
    durations_input_sequence.append(durations_input[note_index])
    
    if len(notes_input_sequence) > seq_len:
        notes_input_sequence = notes_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]

[43, 43, 43, 43, 16]
[0, 0, 0, 0, 7]
0.9999994
5.594128e-07
===
[[9.999994e-01 5.594128e-07]]
[43, 43, 43, 16, 16]
[0, 0, 0, 7, 7]
0.99999976
2.813152e-07
===
[[9.9999976e-01 2.8131521e-07]]
[43, 43, 16, 16, 21]
[0, 0, 7, 7, 10]
0.9999993
7.705501e-07
===
[[9.999993e-01 7.705501e-07]]
[43, 16, 16, 21, 21]
[0, 7, 7, 10, 7]
0.99999845
1.5019099e-06
===
[[9.9999845e-01 1.5019099e-06]]
[16, 16, 21, 21, 25]
[7, 7, 10, 7, 7]
0.9999492
5.0753497e-05
===
[[9.9994922e-01 5.0753497e-05]]
[16, 21, 21, 25, 23]
[7, 10, 7, 7, 7]
0.9999609
3.913069e-05
===
[[9.999609e-01 3.913069e-05]]
[21, 21, 25, 23, 21]
[10, 7, 7, 7, 7]
0.9924648
0.0075351857
===
[[0.9924648  0.00753519]]
[21, 25, 23, 21, 23]
[7, 7, 7, 7, 9]
0.9999211
7.894501e-05
===
[[9.999211e-01 7.894501e-05]]
[25, 23, 21, 23, 23]
[7, 7, 7, 9, 4]
0.9204941
0.07950596
===
[[0.9204941  0.07950596]]
[23, 21, 23, 23, 23]
[7, 7, 9, 4, 7]
0.99983144
0.00016849699
===
[[9.9983144e-01 1.6849699e-04]]
[21, 23, 23, 23, 26]
[7, 9, 4, 7, 7]
0.011743897
0.

In [104]:
print(phrases_output) # unfortunately completely wrong...

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]


In [105]:
print(phrases_input) # should be

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


### Calculate accuracy and F1 score

In [106]:
def accuracy(expected, actual):
    """
    Calculates the accuracy given two arrays of 0s and 1s (expected and actual).
    """
    # Ensure that the inputs are numpy arrays.
    expected = np.array(expected)
    actual = np.array(actual)
    
    # Calculate the number of correct predictions.
    num_correct = np.sum(expected == actual)
    
    # Calculate the total number of predictions.
    num_total = len(expected)
    
    # Calculate the accuracy.
    acc = num_correct / num_total
    
    return acc


def f1_score(expected, actual):
    """
    Calculates the F1 score given two arrays of 0s and 1s (expected and actual).
    """
    # Ensure that the inputs are numpy arrays.
    expected = np.array(expected)
    actual = np.array(actual)
    
    # Calculate the true positives, false positives, and false negatives.
    tp = np.sum((expected == 1) & (actual == 1))
    fp = np.sum((expected == 0) & (actual == 1))
    fn = np.sum((expected == 1) & (actual == 0))
    
    # Calculate the precision, recall, and F1 score.
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    
    return f1

In [107]:
accuracy(phrases_output, phrases_input)

0.8113207547169812

In [108]:
f1_score(phrases_output, phrases_input)

0.16666666666666666